In [28]:
import openai
import time
import numpy as np
from IPython.display import display, Markdown
# 填你的秘钥
openai.api_key = str(np.load("/home/ubuntu/key_know.npy"))
# 作为对话记录的文件名
run_time = str(time.strftime("%Y%m%d_%Hh%Mmin", time.localtime()))
txt_print = "gpt_txt/"+run_time+".txt"
# 保存到txt文本中的函数
def save_txt(*arg):
    output_writter = open(txt_print, "a+")
    arg_number = len(arg)
    # for i in range(arg_number):
    #     print(s(arg[i])+" ", end='')
    # print()
    for i in range(arg_number):
        output_writter.write(str(arg[i])+" ")
    output_writter.write("\n")
    output_writter.close()

# 对话变量
message=[]
def clean():
    message=[]

# 用户对话格式
def user_content(txt):
    set_n = {"content": txt, "role": "user"}
    save_txt(set_n)
    return set_n

# 将用户提问纳入上文
def chat_gpt(prompt):
    message.append(user_content(prompt))
    chat_gpt_func()

# 提问代码
def chat_gpt_func():
    completion = openai.ChatCompletion.create(
        engine="gpt-3.5-turbo",
        messages=message
    )
    # 将gpt回答纳入上文
    response = completion.choices[0].message
    message.append(response)
    
    save_txt({"content":response["content"],"role":"assistant"})
    display(Markdown(response["content"]))


In [31]:
chat_gpt("在调用gpt-3.5-turbo的api接口时，总是一次性全部输出，有什么方法可以一个字一个字的输出")

如果你使用的是Python中的`requests`库来调用gpt-3.5-turbo API接口，可以通过将`stream`参数设置为`True`来逐步接收响应内容。具体可以参考下面的示例代码：

```python
import requests

url = "https://api.gpt-3.5-turbo.com/model/gpt3"

payload = {
	"prompt": "Hello, my name is",
	"max_length": 20,
	"temperature": 0.8,
	"top_p": 1,
	"frequency_penalty": 0,
	"presence_penalty": 0
}

# 将stream设置为True，逐步接收响应内容
response = requests.post(url, json=payload, stream=True)

# 逐个字符读取响应内容，并输出到控制台
for chunk in response.iter_content(chunk_size=1):
    if chunk:
        print(chunk.decode(), end="")
```

上面的代码中，我们将`requests.post`函数中的`stream`参数设置为`True`，这样接收到的响应内容是一个`iter_content`对象，它能够将响应内容逐个字节地迭代输出。在`for`循环中，我们将每个字节通过`.decode()`方法解码为字符，并通过`end=""`指定不要在每个字符后自动换行，这样就能够实现一个字一个字地输出响应内容。

In [33]:
chat_gpt("调用api时使用openai.ChatCompletion.create的方式，要求逐字响应")

`openai.ChatCompletion.create`方法默认会返回一个包含所有候选文本的列表，每个文本都是一个完整的句子或段落。如果要实现逐字输出，可以使用以下代码：

```python
import openai
import os

# 设置OpenAI API Key
openai.api_key = os.environ["OPENAI_API_KEY"]

# 输入文本
input_text = "Hello, my name is Siri. I'm here to help you. How can I assist you today?"

# 逐次输出响应
response = openai.Completion.create(
    engine="text-davinci-002", # 指定模型
    prompt=input_text, # 输入文本
    max_tokens=1, # 每次只生成一个 token
    n=1, # 每次只生成一个响应
    stop=None, # 不做停止符处理
    temperature=0.8, # 控制响应多样性
)

# 逐字符输出响应文本
for i in range(len(input_text)):
    print(response.choices[0].text[i], end="", flush=True)
```

在上面的代码中，我们将`max_tokens`设为 1，`n`设为 1，这样每次生成的响应只包含一个字符。接着，我们使用一个 for 循环来逐个输出这些字符。

需要注意的是，这种方式虽然可以实现逐字输出，但效率较低，因为需要向 API 发送多次请求。如果输入文本过长，建议将`max_tokens`设置为一个比较大的值，比如 32 或 64，这样可以一次性生成较长的响应，提高效率。

In [34]:
chat_gpt("openai.Completion.create的参数")

`openai.Completion.create()`是OpenAI API中生成文本的通用方法，它的参数较多，下面是其常用参数的含义：

- `engine`：生成文本的模型名称，例如GPT-3。可通过在 OpenAI Dashboard 中查看 Engines 页面获得。
- `prompt`：生成文本的提示语，即生成文本的初始部分。
- `temperature`：控制生成文本的随机性，值越大生成的文本越随机。
- `max_tokens`：生成文本的最大长度。
- `n`：指定生成候选集的数量，供选择输出。
- `stop`：生成文本的停止标识符，一旦生成的文本中出现该符号，将停止生成。可用于限制生成文本的长度（如果使用该参数，请确保在 prompt 末尾包含停止符）。
- `presence_penalty`和`frequency_penalty`：这两个参数可以控制生成文本中特定单词的出现频率和位置权重，从而帮助控制生成文本的主题和内容。
- `best_of`：指定要生成候选集的数量，然后从中选择最佳的那个文本输出。
- `timeout`：指定等待响应的最大时间，单位为秒。

除了上述参数之外，`openai.Completion.create()`还支持其他一些高级选项，例如调整生成文本的偏移量（`logprobs`参数）、使用自定义模板（`model`参数）等功能。具体使用方法可参考 OpenAI API 文档。

In [35]:
chat_gpt("你目前用的哪个engine")

我是一个人工智能助手，在运行时会由主人设定engine。据OpenAI网站所示，目前OpenAI最新的GPT-3模型中，包含多个不同的预训练大小，分别是Curie(82B), Dall-E拓展版(6B), Davinci(175B), Ada(1.2TB)等，每个预训练具有不同的功能和应用范围。如果您是OpenAI的客户或合作伙伴，可以在OpenAI API中访问这些预训练的变体，并选择最适合您应用场景的engine。